In [46]:
import numpy as np
import math
import random
t = 0 #time
Th = [] #collector of theta
num_theta = 0 #num theta
A = [] #collector of arm
num_arms = 0 #number of arms
K = 3 #dimension of arm x, y, theta
Y = [] #collector of arm y
num_arms_y = 0 #number of arms of player two
M = [] #world
num_M = 0
M_vero = 0
theta_vero = 0
N = [] #best and second best x
B = [] #best x
sample_correct = [0, 0, 0, 0, 0, 0, 0, 0] #correct at 10, 20, 50, 100, 200, 400, 600, 800

delta = 0.05
sigma = 0.1 #noise

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1


class theta():
    vector = np.array([])
    best_arm = -1
    id

    def __init__(self, param):
        self.vector = param
        global Th, num_theta
        self.id = num_theta
        Th.append(self)
        num_theta +=1
        
class arm():
    arm_vector = np.array([])
    id = 0
    worst_y = -1
    
    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A, N
        A.append(self)
        self.id = num_arms
        num_arms +=1
        N.append(0)
        B.append(0)
                            

    def pull_arm(self, y_index):
        reward = np.dot(self.arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[y_index].arm_vector))) + np.random.normal(0, +sigma)
        Y[y_index].pull_arm(reward, self.id)
        global N
        N[self.id] += 1
        return reward
    
    def find_worst_y(self):
        #print("find worst y")
        mini = 100000000
        index = - 1
        for i in range(0, num_arms_y):
            curr = Y[i].lower_bound(self.arm_vector)
            if curr < mini:
                index = i
                mini = curr
        self.worst_y = index
        #print("mini", mini)
        return mini
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

class arm_y():
    arm_vector = np.array([])
    id = 0
    best_theta = -1
    R = [] #collection of reward
    A_id = [] #id of the i^th arm played
    times = 0
    theta_stimato = [0.0, 0.0, 0.0]

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, Y
        Y.append(self)
        self.id = num_arms_y
        num_arms_y +=1
        
    def pull_arm(self, reward, id_arm):
        self.R.append(reward)
        self.A_id.append(id_arm)
        self.times += 1 
    
    def calcola_A(self):
        res = 0
        for i in range(0,self.times):
            #print("i", i)
            if i == 0:
                #print("i==0, print res", res)
                #print(" A[self.A_id[i]].arm_vector",  A[self.A_id[i]].arm_vector)
                res = np.outer(np.transpose(A[self.A_id[i]].arm_vector), A[self.A_id[i]].arm_vector)
                #print("res", res)
            else:
                #print("i!=0, print i, res", i, res)
                res += np.outer(np.transpose(A[self.A_id[i]].arm_vector), A[self.A_id[i]].arm_vector)
        #print("A", res)
        res += np.diag([1, 1, 1])
        return res

    def calcola_b(self):
        res = []
        global A
        for i in range(0,self.times):
            curr = A[self.A_id[i]].arm_vector
            curr = [self.R[i] * xx for xx in curr]
            if i == 0:
                res = curr
            else:
                res = [x + y for x, y in zip(res, curr)]
        return res
        
    def stima_theta(self):
        if (self.times == 0):
            return self.theta_stimato
        theta_s = np.dot(np.linalg.inv(self.calcola_A()), self.calcola_b())
        self.theta_stimato = theta_s
        return theta_s
    
    def media_reward(self, arm_vector):
        return np.dot(arm_vector, np.transpose(self.stima_theta()))
    
    def calcola_bound(self, arm_vector):
        ris = 2*sigma*math.sqrt(2)*norm_2_mat(arm_vector, np.linalg.inv(self.calcola_A()))*math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(self.times, 2)*K/delta))
        #print("sqrt", math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(self.times, 2)*K/delta)))
        #print("bound", ris)
        return ris
    
    def upper_bound(self, arm_vector):
        return self.media_reward(arm_vector) + self.calcola_bound(arm_vector)
        #return self.media_reward(arm_vector)
    
    def lower_bound(self, arm_vector):
        return self.media_reward(arm_vector) - self.calcola_bound(arm_vector)
        #return self.media_reward(arm_vector)
    
def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

def run():
    global t, A, Y
    t += 1
    #print("TIME", t)
    maxi = - 1000
    for i in range(0, num_arms):
        curr = Y[A[i].worst_y].media_reward(A[i].arm_vector)
        #print("curr", curr)
        if curr > maxi:
            best = i
            maxi = curr
        print("i term and curr value for best, x", i, "y", A[i].worst_y, curr)
    #print("best", best, A[best].worst_y)
    maxi = -1000
    for i in range(0, num_arms):
        if i is not best:
            curr = Y[A[i].worst_y].upper_bound(A[i].arm_vector)
            if curr > maxi:
                second_best = i
                maxi = curr
    #print("second best", second_best, A[second_best].worst_y)
    B[best] += 1
    A[best].pull_arm(A[best].worst_y)
    A[second_best].pull_arm(A[second_best].worst_y)
        
        
        

In [47]:
array = []
for i in range (0, 3):
    array.append(random.random())
a = arm(array)

array = []
for i in range (0, 3):
    array.append(random.random())
b = arm(array)

array = []
for i in range (0, 3):
    array.append(random.random())
c = arm(array)

array = []
for i in range (0, 3):
    array.append(random.random())
d = arm(array)

wor = []
array = []
for i in range (0, 3):
    array.append(random.random())
wor.append(array)
array = []
for i in range (0, 3):
    array.append(random.random())
wor.append(array)
array = []
for i in range (0, 3):
    array.append(random.random())
wor.append(array)
m = world(wor)

array = []
for i in range (0, 3):
    array.append(random.random())
z = arm_y(array)

array = []
for i in range (0, 3):
    array.append(random.random())
k = arm_y(array)

array = []
for i in range (0, 3):
    array.append(random.random())
l = arm_y(array)

print("a", a.arm_vector, "b",b.arm_vector)

a [0.35528457112132783, 0.5170681611014333, 0.731692930670484] b [0.405961351096345, 0.8164470627892454, 0.6558544898946441]


In [48]:

print("a = arm(", a.arm_vector, ")")
print("b = arm(", b.arm_vector, ")")
print("c = arm(", c.arm_vector, ")")
print("d = arm(", d.arm_vector, ")")


print("m = world(", m.matrix, ")")

print("z = arm_y(", z.arm_vector, ")")
print("k = arm_y(", k.arm_vector, ")")
print("l = arm_y(", l.arm_vector, ")")




a = arm( [0.35528457112132783, 0.5170681611014333, 0.731692930670484] )
b = arm( [0.405961351096345, 0.8164470627892454, 0.6558544898946441] )
c = arm( [0.5731908451030495, 0.9872071549081474, 0.7381936689943309] )
d = arm( [0.888299901148066, 0.8947808556267088, 0.8278468675391695] )
m = world( [[0.7251509587442346, 0.770857549512579, 0.22166304052540986], [0.36354417471197054, 0.8341858243493229, 0.4159009332933914], [0.8480793536111114, 0.28724617278877984, 0.9847260738964485]] )
z = arm_y( [0.5188531433852462, 0.8990455826270553, 0.12499263951557493] )
k = arm_y( [0.7557505677634129, 0.7624845042751167, 0.6035711586500424] )
l = arm_y( [0.8779042042655183, 0.8954952459692903, 0.47549511898809427] )
